In [ ]:
import random
import numpy as np

# polarity flipping
def szudzik(x, y):
    return (x * x + x + y) if x >= y else (x + y * y)

# Mask function
def delta(E, Ex):
    return set(random.sample([xi for xi in E if xi not in Ex], len(E)//2))

# Spatial distance function
def fx(x, xi):
    return np.linalg.norm(np.array(x) - np.array(xi))

# Spatial neighbor function
def theta(xi, M, Tx):
    return {x for x in M if fx(x, xi) < Tx}

# Central event function
def find_central_events(E, xi):
    return [e for e in E if e == xi]

# Noise synthesis function
def synthesize_noise(R, Ec, xi, Ti, sigma=0.1, Tt=10):
    noise_events = []
    for x in R:
        p = random.choice([-1, 1])
        t = Ti * (1 + sigma * fx(x, xi))
        if t < Tt:
            noise_events.append({
                'x': x,
                'p': p,
                't': t
            })
    return noise_events

# Polarity mapping
def lambda_mapping(E_tilde):
    encrypted_events = []
    for event in E_tilde:
        # Get the pairing of x and p for each event in E˜
        x, p = event['x'], event['p']
        encrypted_p = p ^ (szudzik(*x) % 2)  # XOR with Szudzik(x) mod 2
        encrypted_events.append({
            'x': x,
            'p': encrypted_p,
            't': event['t']
        })
    return encrypted_events

# Encrypted events
def main_algorithm(E, Ex, Tx, Ti, Tt, sigma=0.1):
    M = delta(E, Ex)
    E_tilde = []

    while M:
        for xi in list(Ex):
            R = theta(xi, M, Tx)

            if not R:
                continue

            Ec = find_central_events(E, xi)

            N = synthesize_noise(R, Ec, xi, Ti, sigma)

            E_tilde.extend(N)

            Ex.update(R)

            M.difference_update(R)

            if not M:
                break

    E_tilde = lambda_mapping(E_tilde)

    return E_tilde

E = set([(x, y) for x in range(10) for y in range(10)])  # 2D grid of events
Ex = set(random.sample(E, 10))  # Random selection of initial events
Tx = 3  # Spatial threshold for neighbors
Ti = 5  # Initial timestamp for the central event
Tt = 10  # Temporal threshold for noise synthesis
sigma = 0.2  # Scaling factor for temporal correlation

# Run the algorithm
result = main_algorithm(E, Ex, Tx, Ti, Tt, sigma)

# Displaying the encrypted event stream
for event in result:
    print(event)
